In [ ]:
from multiprocessing import  Pool
import hashlib
import pandas as pd
import os
from os.path import join
import numpy as np
# from IPython.display import display

In [ ]:
def get_md5checksum(path):
    md5_hash = hashlib.md5()
    a_file = open(path, "rb")
    content = a_file.read()
    md5_hash.update(content)
    digest = md5_hash.hexdigest()
    return md5_hash.hexdigest()


def get_hash(df):
    df['hash'] = df.Name.apply(lambda x: get_md5checksum(join(dirpath, x)))
    return df

def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

# 

In [3]:
df = pd.read_csv('/datasets/ilyas/ChestNets/chestnet_info_data/labels_chest14.csv')
# df = df.loc[:1000]
dirpath = '/datasets/ilyas/ChestNets/images/'

In [4]:
%%time
df = parallelize_dataframe(df, get_hash, 10)

CPU times: user 222 ms, sys: 161 ms, total: 383 ms
Wall time: 2min 50s


In [6]:
df.to_csv('labels_NIH_with_hash.csv', index=0)